# Assignment5: CRM Cleanup @ **DalaShop**
*Files (Ch.10), Exceptions (Ch.10), Unit Tests (Ch.11), and Regular Expressions*  
.....

**Total: 3 points**  (Two questions, 1.5 pts each)  

> This assignment is scenario-based and aligned with Python Crash Course Ch.10 (files & exceptions), Ch.11 (unit testing with `unittest`), and Regular Expressions.

## Scenario
You are a data intern at an online retailer called **DalaShop**.  
Sales exported a **raw contacts** file from the CRM. It contains customer names, emails, and phone numbers, but the formatting is messy and some emails are invalid.  
Your tasks:

1. **Clean** the contacts (Files + Exceptions + Regex).  
2. **Write unit tests** to make sure your helper functions work correctly and keep working in the future.

## Data file (given by the company): `contacts_raw.txt`
Use this exact sample data (you may extend it for your own testing, but do **not** change it when submitting).  
Run the next cell once to create the file beside your notebook.

In [5]:
# Create the provided company dataset file
with open("contacts_raw.txt", "w", encoding="utf-8") as f:
    f.write('Alice Johnson <alice@example.com> , +1 (469) 555-1234\nBob Roberts <bob[at]example.com> , 972-555-777\nSara M. , sara@mail.co , 214 555 8888\n"Mehdi A." <mehdi.ay@example.org> , (469)555-9999\nDelaram <delaram@example.io>, +1-972-777-2121\nNima <NIMA@example.io> , 972.777.2121\nduplicate <Alice@Example.com> , 469 555 1234')
print("Wrote contacts_raw.txt with sample DalaShop data.")

Wrote contacts_raw.txt with sample DalaShop data.


## Q1 (1.5 pts) — CRM cleanup with Files, Exceptions, and Regex
Implement `q1_crm_cleanup.py` to:

1. **Read** `contacts_raw.txt` using `pathlib` and `with`. If the file is missing, **handle** it gracefully with `try/except FileNotFoundError` (print a friendly message; do not crash).
2. **Validate emails** with a simple regex (`r"[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,}"`).  
   - Trim whitespace with `strip()` before checking.  
   - Use **full** matching (not partial).
3. **Normalize phone numbers:** remove all non-digits (e.g., with `re.sub(r"\D", "", raw)`).  
   - If the result has **≥ 10 digits**, keep the **last 10 digits**.  
   - Otherwise, return an **empty string** (`""`).
4. **Filter rows:** keep **only** rows with a valid email.
5. **Deduplicate:** remove duplicates by **email** using **case-insensitive** comparison (e.g., `email.casefold()`). **Keep the first occurrence** and drop later duplicates.
6. **Output CSV:** write to `contacts_clean.csv` with **columns exactly** `name,email,phone` (UTF-8).  
7. **Preserve input order:** the order of rows in `contacts_clean.csv` must match the **first appearance** order from the input file. **Do not sort** the rows.

**Grading rubric (1.5 pts):**
- (0.4) File read/write via `pathlib` + graceful `FileNotFoundError` handling  
- (0.5) Correct email regex validation + filtering  
- (0.4) Phone normalization + case-insensitive de-dup (keep first)  
- (0.2) Clean code, clear names, minimal docstrings/comments

In [6]:
# Write your answer here
from pathlib import Path
import re
import csv

INPUT_FILE = Path("contacts_raw.txt")
OUTPUT_FILE = Path("contacts_clean.csv")

EMAIL_REGEX = r"[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,}"

def normalize_phone(phone_raw):
    """Remove all non-digits, keep last 10 digits if >=10 digits."""
    digits = re.sub(r"\D", "", phone_raw)
    return digits[-10:] if len(digits) >= 10 else ""

def is_valid_email(email_raw):
    """Check if email matches regex after stripping."""
    email = email_raw.strip()
    return re.fullmatch(EMAIL_REGEX, email) is not None

def parse_contact_line(line):
    """Extract name, email, phone from a line."""
    parts = line.strip().split(",")
    name, email, phone = "", "", ""
    if len(parts) == 2:  # name + email/phone combined
        left, right = parts
        match = re.search(r"<(.+?)>", left)
        if match:
            name = left.split("<")[0].strip('" ').strip()
            email = match.group(1).strip()
        else:
            name = left.strip('" ').strip()
            email = right.strip()
        phone = right if not match else right
    elif len(parts) >= 3:
        name = parts[0].strip('" ').strip()
        email = parts[1].strip()
        phone = parts[2].strip()
    else:
        # fallback
        name = parts[0].strip('" ').strip()
    phone = normalize_phone(phone)
    return name, email, phone

def main():
    try:
        lines = INPUT_FILE.read_text(encoding="utf-8").splitlines()
    except FileNotFoundError:
        print(f"Error: {INPUT_FILE} not found. Exiting.")
        return

    cleaned_contacts = []
    seen_emails = set()

    for line in lines:
        name, email, phone = parse_contact_line(line)
        if not is_valid_email(email):
            continue
        email_lower = email.casefold()
        if email_lower in seen_emails:
            continue
        seen_emails.add(email_lower)
        cleaned_contacts.append((name, email, phone))

    # Write CSV
    with OUTPUT_FILE.open("w", newline="", encoding="utf-8") as f:
        writer = csv.writer(f)
        writer.writerow(["name", "email", "phone"])
        writer.writerows(cleaned_contacts)

    print(f"Cleaned contacts written to {OUTPUT_FILE}")

if __name__ == "__main__":
    main()


Cleaned contacts written to contacts_clean.csv


## Q2 (1.5 pts) — Unit testing with `unittest`
Create tests in `test_crm_cleanup.py` that cover at least:

1. **Email validation**: valid/invalid variations.  
2. **Phone normalization**: parentheses, dashes, spaces, country code; too-short cases.  
3. **Parsing**: from a small multi-line string (not from a file), assert the exact structured rows (name/email/phone).  
4. **De-duplication**: demonstrate that a case-variant duplicate email is dropped (first occurrence kept).




In [8]:
# Write your answer here
import unittest
import re

# Copy the helper functions here from Q1

EMAIL_REGEX = r"[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,}"

def normalize_phone(phone_raw):
    digits = re.sub(r"\D", "", phone_raw)
    return digits[-10:] if len(digits) >= 10 else ""

def is_valid_email(email_raw):
    email = email_raw.strip()
    return re.fullmatch(EMAIL_REGEX, email) is not None

def parse_contact_line(line):
    parts = line.strip().split(",")
    name, email, phone = "", "", ""
    if len(parts) == 2:
        left, right = parts
        match = re.search(r"<(.+?)>", left)
        if match:
            name = left.split("<")[0].strip('" ').strip()
            email = match.group(1).strip()
        else:
            name = left.strip('" ').strip()
            email = right.strip()
        phone = right if not match else right
    elif len(parts) >= 3:
        name = parts[0].strip('" ').strip()
        email = parts[1].strip()
        phone = parts[2].strip()
    phone = normalize_phone(phone)
    return name, email, phone

# Unit tests
class TestCRMHelpers(unittest.TestCase):
    def test_email_validation(self):
        self.assertTrue(is_valid_email("alice@example.com"))
        self.assertTrue(is_valid_email("Bob.Roberts+1@mail.co"))
        self.assertFalse(is_valid_email("bob[at]example.com"))
        self.assertFalse(is_valid_email("invalid-email@"))
        self.assertFalse(is_valid_email("  "))

    def test_phone_normalization(self):
        self.assertEqual(normalize_phone("(469) 555-1234"), "4695551234")
        self.assertEqual(normalize_phone("+1-972-777-2121"), "9727772121")
        self.assertEqual(normalize_phone("972.777.2121"), "9727772121")
        self.assertEqual(normalize_phone("214 555 8888"), "2145558888")
        self.assertEqual(normalize_phone("1234"), "")

    def test_parse_contact_line(self):
        line = 'Alice Johnson <alice@example.com> , +1 (469) 555-1234'
        name, email, phone = parse_contact_line(line)
        self.assertEqual(name, "Alice Johnson")
        self.assertEqual(email, "alice@example.com")
        self.assertEqual(phone, "4695551234")

        line2 = 'Sara M. , sara@mail.co , 214 555 8888'
        name2, email2, phone2 = parse_contact_line(line2)
        self.assertEqual(name2, "Sara M.")
        self.assertEqual(email2, "sara@mail.co")
        self.assertEqual(phone2, "2145558888")

    def test_deduplication(self):
        lines = [
            'Alice Johnson <alice@example.com> , +1 (469) 555-1234',
            'duplicate <Alice@Example.com> , 469 555 1234'
        ]
        seen_emails = set()
        cleaned = []
        for line in lines:
            name, email, phone = parse_contact_line(line)
            if not is_valid_email(email):
                continue
            email_lower = email.casefold()
            if email_lower in seen_emails:
                continue
            seen_emails.add(email_lower)
            cleaned.append((name, email, phone))
        self.assertEqual(len(cleaned), 1)
        self.assertEqual(cleaned[0][0], "Alice Johnson")

if __name__ == "__main__":
    unittest.main(argv=[''], exit=False)


....
----------------------------------------------------------------------
Ran 4 tests in 0.004s

OK


## Grading rubric (total 3 pts)
- **Q1 (1.5 pts)**  
  - (0.4) File I/O with `pathlib` + graceful `FileNotFoundError` handling  
  - (0.5) Email validation (regex + strip + full match) and filtering  
  - (0.4) Phone normalization and **case-insensitive** de-duplication (keep first)  
  - (0.2) Code clarity (names, minimal docstrings/comments)
- **Q2 (1.5 pts)**  
  - (0.6) Meaningful coverage for email/phone functions (valid & invalid)  
  - (0.6) Parsing & de-dup tests that assert exact expected rows  
  - (0.3) Standard `unittest` structure and readable test names
